In [1]:
import pathlib
import os
import cv2
import matplotlib.pyplot as plt
import time
import numpy as np
import pandas as pd
import scipy.ndimage as ndimage
import tensorflow as tf
from tensorflow.keras.layers import Dense,GlobalMaxPooling2D,Dropout,Flatten,BatchNormalization,Activation
from tensorflow.keras.applications import NASNetLarge,VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import imagenet_utils
from tensorflow.keras.applications.nasnet import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array,load_img,ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K

In [2]:
tf.__version__

'2.4.1'

In [3]:
anime_folder = './Anime/*.jpg'
human_folder = './Human/*.png'
IMAGE_SIZE = [256, 256]
VALIDATION_SPLIT = 0.3
batch_size1 = 15

In [4]:
anime = tf.io.gfile.glob(anime_folder)
split = int(len(anime) * VALIDATION_SPLIT)
training_filenames = anime[split:]
validation_filenames = anime[:split]
print("Pattern matches {} data files. Splitting dataset into {} training files and {} validation files".format(len(anime), len(training_filenames), len(validation_filenames)))
validation_steps = int(len(validation_filenames)/ batch_size1)
steps_per_epoch = int(len(training_filenames)/ batch_size1)
print("With a batch size of {}, there will be {} batches per training epoch and {} batch(es) per validation run.".format(batch_size1, steps_per_epoch, validation_steps))

Pattern matches 92257 data files. Splitting dataset into 64580 training files and 27677 validation files
With a batch size of 15, there will be 4305 batches per training epoch and 1845 batch(es) per validation run.


In [5]:
human = tf.io.gfile.glob(human_folder)
split1 = int(len(human) * VALIDATION_SPLIT)
training_filenames1 = human[split1:]
validation_filenames1 = human[:split1]
print("Pattern matches {} data files. Splitting dataset into {} training files and {} validation files".format(len(human), len(training_filenames1), len(validation_filenames1)))
validation_steps = int(len(validation_filenames1)/ batch_size1)
steps_per_epoch = int(len(training_filenames1)/ batch_size1)
print("With a batch size of {}, there will be {} batches per training epoch and {} batch(es) per validation run.".format(batch_size1, steps_per_epoch, validation_steps))

Pattern matches 21754 data files. Splitting dataset into 15228 training files and 6526 validation files
With a batch size of 15, there will be 1015 batches per training epoch and 435 batch(es) per validation run.


In [6]:
resized=[]
for i in range(len(human)):
    img = cv2.imread(human[i], cv2.IMREAD_UNCHANGED)
    resized.append(cv2.resize(img, (256,256), interpolation = cv2.INTER_AREA))
    status = cv2.imwrite('./Human_resized/' + str(i) + '.jpg',resized[i])

In [7]:
resized=[]
for i in range(len(anime)):
    img = cv2.imread(anime[i], cv2.IMREAD_UNCHANGED)
    resized.append(cv2.resize(img, (256,256), interpolation = cv2.INTER_AREA))
    status = cv2.imwrite('./Anime_resized/' + str(i) + '.jpg',resized[i])

In [8]:
human_folder_resized = './Human_resized/*.jpg'
human_resized = tf.io.gfile.glob(human_folder_resized)

In [10]:
anime_folder_resized = './Anime_resized/*.jpg'
anime_resized = tf.io.gfile.glob(anime_folder_resized)

In [11]:
dic_human = {}
for i in range(len(human_resized)):
    dic_human[human_resized[i]]  =  i

In [12]:
dic_anime = {}
for i in range(len(anime_resized)):
    dic_anime[anime_resized[i]]  =  i

In [13]:
def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [14]:
def image_example(image_string, label):
    image_shape = tf.image.decode_jpeg(image_string).shape
    
    feature = {
      'label': _int64_feature(label),
      'image_raw': _bytes_feature(image_string),
  }

    return tf.train.Example(features=tf.train.Features(feature=feature))

In [15]:
anime_file = 'CycleGANime-Anime.tfrecords'

In [16]:
human_file = 'CycleGANime-Humans.tfrecords'

In [17]:
with tf.io.TFRecordWriter(anime_file) as writer:
    for filename, label in dic_anime.items():
        image_string = open(filename, 'rb').read()
        tf_example = image_example(image_string, label)
        writer.write(tf_example.SerializeToString())

In [18]:
with tf.io.TFRecordWriter(human_file) as writer:
    for filename, label in dic_human.items():
        image_string = open(filename, 'rb').read()
        tf_example = image_example(image_string, label)
        writer.write(tf_example.SerializeToString())